## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.json("/user/sprenner/lookup_table_2_official.json")
df_lookup.createOrReplaceTempView("lookup")

In [5]:
df_april = spark.read.json("/user/sprenner/training_official_april_hourly.json")
df_april = df_april.withColumn("join_key_april", concat_ws(':', df_april["scope"], df_april["filename"]))
df_april = df_april.withColumn("fileScope", df_april["scope"])
df_april = df_april.drop("scope")
df_april.createOrReplaceTempView("april")

In [6]:
df_lookup.printSchema()

root
 |-- CHILD_NAME: string (nullable = true)
 |-- CHILD_SCOPE: string (nullable = true)
 |-- DID_TYPE: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- SCOPE: string (nullable = true)
 |-- join_key_lookup: string (nullable = true)



In [7]:
df_april.count()

777286

In [8]:
df_joined = df_april.join(df_lookup, df_april.join_key_april == df_lookup.join_key_lookup, "left_outer")

In [9]:
df_joined.count()

777286

In [10]:
df_joined.createOrReplaceTempView("joined")

In [11]:
spark.sql("SELECT * FROM joined WHERE join_key_lookup IS NULL").toPandas()
#waren bei non officials 3808 rows
#bei officials 3168 - evtl mehr weil durch das filtern weniger doppelte datasets rausgeschmissen wurden

account  accountHash         clientState  clientStateHash  \
0     ellingha -171710354    DONE               -878527175         
1     ellingha -171710354    DONE               -878527175         
2     tamartin  1971112356   DONE               -878527175         
3     toyamaza  368790896    DONE               -878527175         
4     tbold    -1107852510   DONE               -878527175         
5     hepikhar -1532482556   DONE               -878527175         
6     hepikhar -1532482556   DONE               -878527175         
7     ellingha -171710354    DONE               -878527175         
8     ellingha -171710354    DONE               -878527175         
9     ellingha -171710354    DONE               -878527175         
10    ellingha -171710354    DONE               -878527175         
11    tamartin  1971112356   DONE               -878527175         
12    tamartin  1971112356   DONE               -878527175         
13    tamartin  1971112356   DONE               -878527175         
14    toyamaza  368790896    DONE               -878527175         
15    toyamaza  368790896    DONE               -878527175         
16    toyamaza  368790896    DONE               -878527175         
17    lhavener  1677896849   DONE               -878527175         
18    tbold    -1107852510   DONE               -878527175         
19    hatmani  -1973289280   ServiceUnavailable -1394468120        
20    hatmani  -1973289280   ServiceUnavailable -1394468120        
21    hatmani  -1973289280   ServiceUnavailable -1394468120        
22    hatmani  -1973289280   ServiceUnavailable -1394468120        
23    hatmani  -1973289280   ServiceUnavailable -1394468120        
24    hatmani  -1973289280   ServiceUnavailable -1394468120        
25    hatmani  -1973289280   ServiceUnavailable -1394468120        
26    hatmani  -1973289280   DONE               -878527175         
27    ellingha -171710354    DONE               -878527175         
28    ellingha -171710354    DONE               -878527175         
29    ellingha -171710354    DONE               -878527175         
...        ...        ...     ...                      ...         
3138  toyamaza  368790896    DONE               -878527175         
3139  toyamaza  368790896    DONE               -878527175         
3140  toyamaza  368790896    DONE               -878527175         
3141  toyamaza  368790896    DONE               -878527175         
3142  toyamaza  368790896    DONE               -878527175         
3143  toyamaza  368790896    DONE               -878527175         
3144  tbold    -1107852510   DONE               -878527175         
3145  sbartkow  680505234    DONE               -878527175         
3146  ellingha -171710354    DONE               -878527175         
3147  ellingha -171710354    DONE               -878527175         
3148  ellingha -171710354    DONE               -878527175         
3149  ellingha -171710354    DONE               -878527175         
3150  ellingha -171710354    DONE               -878527175         
3151  ellingha -171710354    DONE               -878527175         
3152  ellingha -171710354    DONE               -878527175         
3153  ellingha -171710354    DONE               -878527175         
3154  toyamaza  368790896    DONE               -878527175         
3155  lhavener  1677896849   DONE               -878527175         
3156  tbold    -1107852510   DONE               -878527175         
3157  tbold    -1107852510   DONE               -878527175         
3158  sswift    362219909    DONE               -878527175         
3159  hepikhar -1532482556   DONE               -878527175         
3160  ellingha -171710354    DONE               -878527175         
3161  ellingha -171710354    DONE               -878527175         
3162  tamartin  1971112356   DONE               -878527175         
3163  tamartin  1971112356   DONE               -878527175         
3164  tamartin  1971112356   DONE               -878527175     

In [12]:
df_joined = df_joined.withColumn('dataset_lookup',when(col("join_key_lookup").isNull(), df_joined["dataset"]).otherwise(df_joined["NAME"]))

## create new dataset column split for updated datasets, retain old ones (where join key is null)

In [13]:
df_joined.describe().toPandas()

summary  account           accountHash         clientState  \
0  count   777286   777286                777286               
1  mean    None     1.935684591732567E8   None                 
2  stddev  None     1.1361317241087067E9  None                 
3  min     aagaard  -2127257360           CannotAuthenticate   
4  max     zzheng   2143592175            SourceNotFound       

        clientStateHash  \
0  777286                 
1  -9.74016245191591E8    
2  2.6212986830087784E8   
3  -1922357674            
4  1269532611             

                                                                                               dataset  \
0  777286                                                                                                
1  None                                                                                                  
2  None                                                                                                  
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456                                               
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210   

            datasetHash datasetScope      datasetScopeHash dataset_version  \
0  777286                777286       777286                777286           
1  1.4554549800768314E7  None         -7.552649492442358E8  None             
2  1.1941649925847783E9  None         8.314865501449608E8   None             
3  -2147401420           data10_hi    -2057006107           c1130_m1847      
4  2147445885            mc16_valid   2109319881            x557_m1947       

    dataset_versionHash datatype          datatypeHash eventType  \
0  777286                777286   777286                777286     
1  2.973310495739393E8   None     4.947060556405943E8   None       
2  1.2288188131163635E9  None     1.1700303292632782E9  None       
3  -2146716518           AOD      -2070622869           download   
4  2146405934            log      2124512252            download   

  eventTypeHash eventVersion      eventVersionHash  \
0  777286        777286       777286                 
1  5.937808E7    None         -1.992468944589716E9   
2  0.0           None         3.073273964746103E8    
3  59378080      1.13.2       -2044360925            
4  59378080      1.15.5       1261702107             

                             filename          filenameHash  \
0  777286                              777286                 
1  None                                -248557.46481861244    
2  None                                1.2402658149710774E9   
3  AOD.05371635._000001.pool.root.1    -2147478094            
4  log.13787275._000072.job.log.tgz.1  2147482617             

        filesize_double                hits                  hostname  \
0  777286                777286              777286                     
1  6.11402028756827E8    2.010655022732945   None                       
2  1.2408771809945784E9  2.3375900926294975  None                       
3  304.0                 1                   ReCaS-ui-01.cs.infn.it     
4  1.1991446515E10       40                  xenia.nevis.columbia.edu   

            hostnameHash           hour               hourHash  \
0  777286                 777286         777286                  
1  -1.1804844919942595E8  None           -2.5176970566176414E7   
2  1.2286452227051134E9   None           1.2215909376898785E9    
3  -2133418459            2018-04-01 00  -2138243719             
4  2129595856             2018-04-30 23  2144044799              

                                        join_key_1        join_key_1Hash  \
0  777286                                           777286                 
1  None                                             -1253427.7464974283    
2  None                                             1.2404336000298734E9   
3  AOD.05371635._000001.pool.root.12018-04-02 11    -2147482159            
4  log.13787275._00

In [14]:
df_joined_dropped = df_joined.drop('dataset','datasetHash','prod_step','prod_stepHash','project','projectHash','run_number','run_numberHash','stream_name','stream_nameHash','datatype','datatypeHash','dataset_version','dataset_versionHash')

In [15]:
split_col = split(df_joined_dropped['dataset_lookup'], "\.")
df_joined_dropped = df_joined_dropped.withColumn('project', split_col.getItem(0))
df_joined_dropped = df_joined_dropped.withColumn('run_number', split_col.getItem(1))
df_joined_dropped = df_joined_dropped.withColumn('stream_name', split_col.getItem(2))
df_joined_dropped = df_joined_dropped.withColumn('prod_step', split_col.getItem(3))
df_joined_dropped = df_joined_dropped.withColumn('datatype', split_col.getItem(4))
df_joined_dropped = df_joined_dropped.withColumn('dataset_version', split_col.getItem(5))

In [16]:
df_joined_dropped.count()

777286

In [17]:
df_joined_dropped = df_joined_dropped.withColumn("dataset", df_joined_dropped['dataset_lookup'])
df_joined_dropped = df_joined_dropped.drop("dataset_lookup")

In [18]:
categoricalFeatures = ['dataset', 'project', 'run_number', 'stream_name', 'prod_step', 'datatype', 'dataset_version']

encoded = df_joined_dropped

for elem in categoricalFeatures:
    encoded = encoded.withColumn(elem + "Hash", hash(encoded[elem]))

In [22]:
encoded = encoded.drop('NAME', 'SCOPE', 'CHILD_NAME', 'CHILD_SCOPE','DID_TYPE','join_key_lookup','join_key_april','join_key_1', 'join_key_1Hash')

In [23]:
encoded.describe().toPandas()

summary  account           accountHash         clientState  \
0  count   777286   777286                777286               
1  mean    None     1.935684591732567E8   None                 
2  stddev  None     1.1361317241087067E9  None                 
3  min     aagaard  -2127257360           CannotAuthenticate   
4  max     zzheng   2143592175            SourceNotFound       

       clientStateHash datasetScope      datasetScopeHash eventType  \
0  777286               777286       777286                777286     
1  -9.74016245191591E8  None         -7.552649492442358E8  None       
2  2.621298683008779E8  None         8.3148655014496E8     None       
3  -1922357674          data10_hi    -2057006107           download   
4  1269532611           mc16_valid   2109319881            download   

  eventTypeHash eventVersion      eventVersionHash  \
0  777286        777286       777286                 
1  5.937808E7    None         -1.992468944589716E9   
2  0.0           None         3.0732739647461027E8   
3  59378080      1.13.2       -2044360925            
4  59378080      1.15.5       1261702107             

                             filename          filenameHash  \
0  777286                              777286                 
1  None                                -248557.46481861244    
2  None                                1.2402658149710774E9   
3  AOD.05371635._000001.pool.root.1    -2147478094            
4  log.13787275._000072.job.log.tgz.1  2147482617             

       filesize_double                hits                  hostname  \
0  777286               777286              777286                     
1  6.11402028756827E8   2.010655022732945   None                       
2  1.240877180994578E9  2.3375900926294975  None                       
3  304.0                1                   ReCaS-ui-01.cs.infn.it     
4  1.1991446515E10      40                  xenia.nevis.columbia.edu   

            hostnameHash           hour               hourHash protocol  \
0  777286                 777286         777286                 777286    
1  -1.1804844919942595E8  None           -2.5176970566176414E7  None      
2  1.2286452227051132E9   None           1.2215909376898792E9   None      
3  -2133418459            2018-04-01 00  -2138243719            davs      
4  2129595856             2018-04-30 23  2144044799             srm       

           protocolHash             remoteSite        remoteSiteHash  \
0  777286                777286                 777286                 
1  3.5590511037907284E8  None                   -6.188806654015021E8   
2  1.0536208601928816E9  None                   1.2385341645403852E9   
3  -1607963002           AGLT2_CALIBDISK        -2113133296            
4  2128034789            ZA-WITS-CORE_PRODDISK  2141740830             

              scopeHash                           traceId  \
0  777286                777286                             
1  -7.569339557869073E8  None                               
2  8.310816593895098E8   None                               
3  -2057006107           00001963ee5a49178eccc3cd255fcb72   
4  2109319881            fffff6db150d4a178afa9a39131df4fd   

            traceIdHash                    traceIp           traceIpHash  \
0  777286                777286                     777286                 
1  -1142514.935878171    None                       -3.101995473497927E8   
2  1.2399043565823288E9  None                       1.2291202980313933E9   
3  -2147481929           2001:1458:201:e4::100:28c  -2143850866            
4  2147480674            ::ffff:95.103.58.33        2120978876             

     traceTimeentryUnix         transferStart  \
0  777286                777286                 
1  1.5236944545257301E9  1.5236943924140377E9   
2  652927.2270174315     652914.6027116699      
3  1.522533840149492E9   1.522527378084161E9    
4  1.525125586283086E9   1.525125342411194E9    

                               uuid               

In [ ]:
#TODO: beim nächsten mal verwenden mal schauen ob hier sortieren noch was ändert
encoded.limit(200).toPandas()

In [ ]:
encoded = encoded.sort("traceTimeentryUnix", ascending=True)

In [ ]:
encoded.limit(200).toPandas()

In [24]:
#encoded.write.format("json").save("/user/sprenner/training_official_april_hourly_cleaned_datasets.json")